# HUK Coburg Coding Challenge

## 01 | Setting up the environment

In [46]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

DATA_D01_FOLDER_PATH = "data/d01/"

df_per = pd.read_csv(os.path.join(DATA_D01_FOLDER_PATH, "alter_geschlecht.csv"))
df_int = pd.read_csv(os.path.join(DATA_D01_FOLDER_PATH, "interesse.csv"))
df_res = pd.read_csv(os.path.join(DATA_D01_FOLDER_PATH, "rest.csv"), sep = ";")

print("Personal data shape: ", df_per.shape)
print("Interest data shape: ", df_int.shape) # -> less observations
print("Rest data shape: ", df_res.shape)

Personal data shape:  (508146, 3)
Interest data shape:  (381109, 2)
Rest data shape:  (508146, 9)


| Variable Name   | Type       | Description                                                                 | Prior Assumption                                                                                   |
|-----------------|------------|-----------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|
| ID              | Integer    | Eindeutige ID des Kunden                                                    | Keine direkte Auswirkung auf Interesse                                                             |
| Geschlecht      | Categorical| Geschlecht des Kunden (z.B. männlich/weiblich)                              | Mögliche Unterschiede in der Affinität je nach Geschlecht                                           |
| Alter           | Integer    | Alter des Kunden in Jahren                                                  | Jüngere oder ältere Altersgruppen könnten unterschiedlich interessiert sein                        |
| Fahrerlaubnis   | Binary     | Ob der Kunde eine Fahrerlaubnis besitzt (Ja=1, Nein=0)                      | Kunden ohne Fahrerlaubnis zeigen vermutlich kein Interesse                                         |
| Regional_Code   | Float      | Eindeutiger Code der Wohnregion des Kunden                                  | Regionale Unterschiede könnten das Interesse beeinflussen                                          |
| Vorversicherung | Binary     | Ob der Kunde bereits eine Kfz-Versicherung hat (Ja=1, Nein=0)               | Kunden ohne Vorversicherung könnten eher interessiert sein                                         |
| Alter_Fzg       | String     | Alter des Fahrzeugs des Kunden ("1-2 Year" etc.)                            | Besitzer neuerer Fahrzeuge könnten eher Interesse zeigen                                           |
| Vorschaden      | Binary     | Ob der Kunde bereits einen Schaden am Fahrzeug hatte (Yes, No)              | Kunden mit Vorschäden könnten verstärktes Interesse an einem besseren Angebot haben                |
| Jahresbeitrag   | Float      | Erwarteter Jahresbeitrag in Euro bei Abschluss der Versicherung             | Höhere Beiträge könnten das Interesse negativ beeinflussen                                         |
| Vertriebskanal  | Float      | Code für den genutzten Vertriebskanal                                       | Bestimmte Vertriebskanäle könnten erfolgreicher im Wecken des Interesses sein                      |
| Kundentreue     | Integer    | Anzahl der Tage, seit denen eine Kundenbeziehung besteht                    | Längere Kundentreue könnte mit höherem Interesse korrelieren                                       |
| Interesse       | Binary     | Zielvariable: Ob der Kunde an einem Versicherungsangebot interessiert ist (Ja=1, Nein=0) | -                                                                                     |

**Checks:**
- ✅ All data sets only contain unique IDs
- ❓ Interest dataframe has less rows than the other dataframes --> assumption: missing values are test data since max interest id is 381109 (= length of interest dataframe)
- ✅ Otherwise no missing values in any dataframe

## 02 | Prepare the Data

In [56]:
# Create dataframe that contains all the data (except the label)
df = pd.merge(df_per, df_res, on = "id")
df

# Create the "test" data set (needed for validation whether train and test data "look alike")
# df_test = pd.merge(df_per, df_res, on = "id")
# df_test = df_test[~df_test.id.isin(df.id)]
# df_test

,id,Geschlecht,Alter,Fahrerlaubnis,Regional_Code,Vorversicherung,Alter_Fzg,Vorschaden,Jahresbeitrag,Vertriebskanal,Kundentreue
0,170727,Male,42,1,8.0,0,1-2 Year,No,27324.0,152.0,208
1,41795,Male,24,1,39.0,0,< 1 Year,Yes,25182.0,152.0,258
2,369321,Female,30,1,36.0,0,< 1 Year,Yes,35700.0,152.0,172
3,500463,Female,32,1,46.0,0,< 1 Year,Yes,23389.0,124.0,110
4,327695,Male,34,1,36.0,1,1-2 Year,No,31551.0,26.0,18
...,...,...,...,...,...,...,...,...,...,...,...
508141,180256,Male,29,1,33.0,1,< 1 Year,No,45381.0,152.0,134
508142,233153,Male,29,1,28.0,1,< 1 Year,No,36152.0,152.0,156
508143,53459,Female,35,1,36.0,1,1-2 Year,No,24619.0,151.0,96
508144,494977,Female,45,1,31.0,0,1-2 Year,Yes,2630.0,30.0,76


## 03 | Descriptive Statistics